<a href="https://colab.research.google.com/github/kfklaihk/Works/blob/main/Generate_Script_From_Config.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

def process_hash_config(file_path):
    # Read the CSV file (case insensitive headers)
    df = pd.read_csv(file_path)
    df.columns = df.columns.str.lower()  # Convert headers to lowercase

    # Create the four dataframes with updated fields
    df1 = df[['table', 'field', 'include_in_select']].drop_duplicates()
    df2 = df[['table', 'field', 'translate_after_merge']].drop_duplicates()
    df3 = df[['table', 'cross_apply', 'cross_apply_field_name', 'left_join', 'left_join_field_name','left_join_where','hash_method']].drop_duplicates()
    df4 = df[['table', 'field', 'hash_method', 'sub_method', 'arg', 'update_after_merge']].drop_duplicates()
    df5 = df[['table', 'update_where', 'merge_where']].drop_duplicates()
    df6 = df[['table', 'update_after_where']].drop_duplicates()
    # Print dataframes for checking
    print("\nDataframe 1 (Table, Field):")
    print(df1)
    print("\nDataframe 2 (Table, Field, Translate_after_merge):")
    print(df2)
    print("\nDataframe 3 (Table, Cross_apply, Cross_apply_Field_Name, Left_join, Left_join_Field_Name,Left_join_where, Hash_Method):")
    print(df3)
    print("\nDataframe 4 (Table, Field, Hash_Method, Sub_Method, Arg, Update_after_merge):")
    print(df4)
    print("\nDataframe 5 (Table, Update_where, Merge_where):")
    print(df5)
    print("\nDataframe 6 (Table, Update_after_where):")
    print(df6)
    # Process each unique table
    unique_tables = df1['table'].unique()
    list1 = []
    list1_1=[]
    list2 = []
    list3 = []
    list4 = []
    list5 = []
    list6 = []      # New list for update_after_merge = Y
    list7 = []
    list8 = []
    list9 = []
    list11 = []

    for table in unique_tables:
        # List 1: Modified to check Include_in_Select
        fields = df1[(df1['table'] == table) & (df1['include_in_select'] == 'Y')]['field'].apply(lambda x: f"org.{x}").tolist()
        cross_apply_fields = df3[df3['table'] == table]['cross_apply_field_name'].dropna().unique().tolist()
        left_join_fields = df3[df3['table'] == table]['left_join_field_name'].dropna().unique().tolist()
        all_fields = ['org.ID'] + fields + cross_apply_fields + left_join_fields
       # all_fields_1 = left_join_fields
        list1.append((table, all_fields))
       # list1_1.append((table, all_fields_1))

        # List 2: Create tuples with table and constructed SQL
        table_df3 = df3[df3['table'] == table]
        sql_texts = []
        # Group by cross_apply_field_name and hash_method
        grouped = table_df3.groupby(['cross_apply_field_name', 'hash_method'])

        for (field_name, hash_method), group in grouped:

            if hash_method == 'fn_dc_method11_hash':
                # Concatenate all cross_apply fields with ||
                cross_applies = [f"org.{x}" for x in group['cross_apply']]
                concatenated = " || ".join(cross_applies)
                s1 = f"CONCAT(m11, {concatenated})"
                sql = f"SELECT fn_dc_method11_hash({s1}, m11) {field_name} FROM dual"
                sql_texts.append(sql)
        if sql_texts:
            list2.append((table, sql_texts))
        # Group by left_join_field_name and hash_method
        sql_texts = []
        grouped_left = table_df3.groupby(['left_join_field_name', 'hash_method'])

        for (field_name, hash_method), group in grouped_left:
            if hash_method == 'Method5':
                # For Method5, process each left_join separately
                for index, (i,row) in enumerate(group.iterrows()):
                    left_join = f"org.{row['left_join']}"
                    left_join_where = f"org.{row['left_join_where']}"
                    left_join_where_flag = f"and {left_join_where}" if pd.notna(row['left_join_where']) else None
                    sql = f"""SELECT map.randID {field_name}
                    FROM dm_mapping_list_hash map
                    WHERE SUBSTR({left_join}, -6) = map.sourceID \n            {left_join_where_flag if left_join_where_flag else ''}"""
                    sql_texts.append(sql)

        if sql_texts:
            list3.append((table, sql_texts))

        #if sql_texts2:
        #    list3.append((table, sql_texts2))

        # List 4: Modified to check Update_after_Merge is blank
        table_df4 = df4[(df4['table'] == table) & (df4['update_after_merge'].isna())]
        sql_texts4 = []

        for _, row in table_df4.iterrows():
            if table_df3[table_df3['table']==table].drop(columns=['hash_method']).drop(columns=['table']).isna().all().all():
                field = f"dst.{row['field']}"
                arg = f"dst.{row['arg']}" if pd.notna(row['arg']) else None
            else:
                field = f"dst.{row['field']}"
                arg = f"src.{row['arg']}" if pd.notna(row['arg']) else None



            if row['hash_method'] == 'fn_dc_method11_hash':
                if row['sub_method'] == 'method1':
                    sql = f"{field} = CASE WHEN {field} IS NOT NULL THEN SUBSTR({arg}, 1, 8) ELSE NULL END"
                elif row['sub_method'] == 'method2':
                    sql = f"{field} = CASE WHEN {field} IS NOT NULL THEN SUBSTR({arg}, 9, 8) ELSE NULL END"
                elif row['sub_method'] == 'method3':
                    sql = f"{field} = CASE WHEN {field} IS NULL THEN NULL ELSE SUBSTR({arg}, 1, 8) || ' ' || SUBSTR({arg}, 9, 8) END"
                elif row['sub_method'] == 'method4':
                    sql = f"{field} = CASE WHEN {field} IS NULL THEN NULL ELSE SUBSTR({arg}, 1, 16) END"

            elif row['hash_method'] == 'Method5':
                sql = f"{field} = CASE WHEN {field} IS NULL THEN NULL WHEN {field} in ('NOT PROVIDED' , 'N/A') THEN {field} ELSE CONCAT(SUBSTR({field}, 1, LENGTH({field}) - LENGTH(SUBSTR({field}, -6))), COALESCE({arg}, SUBSTR({field}, -6))) END"

            elif row['hash_method'] == 'fn_dc_method10_passport_hash':
                if pd.isna(row['sub_method']):
                    sql = f"""{field} = CASE WHEN {field} IS NULL THEN NULL
                                        WHEN {field} in ('NOT PROVIDED' , 'N/A') THEN cast({field} as nvarchar2(100))
                                        ELSE fn_dc_method10_passport_hash({field})
                                        END"""
                elif row['sub_method'] == 'method1':
                    sql = f"""{field} = CASE
                        WHEN {field} like '%-PASSPORT-%' THEN CONCAT(substr({field},1, instr({field},'-PASSPORT-')+9) , fn_dc_method10_passport_hash(substr({field},instr({field},'-PASSPORT-')+10,99)))
                        WHEN SUBSTR({field}, 1, 9) = 'PASSPORT-' THEN CONCAT(SUBSTR({field}, 1, 9), fn_dc_method10_passport_hash(SUBSTR({field}, 10, LENGTH({field})-9)))
                        WHEN SUBSTR({field}, 1, 5) = 'HKID-' THEN CONCAT(SUBSTR({field}, 1, 5), fn_dc_method10_passport_hash(SUBSTR({field}, 6, LENGTH({field})-5)))
                        WHEN {field} like '%HKID-%' THEN substr({field},0, instr({field},'HKID-')+4) || rtrim(fn_dc_method10_passport_hash(substr({field}, instr({field},'HKID-')+5,6)))||
                            substr(use_customer_no, instr({field},'HKID-')+11,99)
                        WHEN {field} in ('NOT PROVIDED' , 'N/A') THEN cast({field} as nvarchar2(100))
                        ELSE fn_dc_method10_passport_hash({field})
                        END"""
                elif row['sub_method'] == 'method2':
                    sql = f"""{field} = CASE
                        WHEN {field} like '%-PASSPORT-%' THEN CONCAT(substr({field},1, instr({field},'-PASSPORT-')+9) , fn_dc_method10_passport_hash(substr({field},instr({field},'-PASSPORT-')+10,99)))
                        WHEN SUBSTR({field}, 1, 9) = 'PASSPORT-' THEN CONCAT(SUBSTR({field}, 1, 9), fn_dc_method10_passport_hash(SUBSTR({field}, 10, LENGTH({field})-9)))
                        WHEN SUBSTR({field}, 1, 5) = 'HKID-' THEN CONCAT(SUBSTR({field}, 1, 5), fn_dc_method10_passport_hash(SUBSTR({field}, 6, LENGTH({field})-5)))
                        WHEN {field} like '%HKID-%' THEN substr({field},0, instr({field},'HKID-')+4) || rtrim(fn_dc_method10_passport_hash(substr({field}, instr({field},'HKID-')+5,6)))||
                        substr(use_customer_no, instr({field},'HKID-')+11,99)
                        WHEN {field} like '%BR-%' Then substr({field}, 0,  instr({field},'BR-')+2)||rtrim(fn_dc_method10_passport_hash(substr({field},instr({field},'BR-')+3,99)))
			            WHEN {field} like '%OTHERS-%' Then substr({field}, 0,  instr({field},'OTHERS-')+6)||rtrim(fn_dc_method10_passport_hash(substr({field},instr({field},'OTHERS-')+7,99)))
			            WHEN {field} like '%SO-%' Then substr({field}, 0,  instr({field},'SO-')+2)||rtrim(fn_dc_method10_passport_hash(substr({field},instr({field},'SO-')+3,99)))
		                WHEN {field} like '%IRD-%' Then substr({field}, 0,  instr({field},'IRD-')+3)||rtrim(fn_dc_method10_passport_hash(substr({field},instr({field},'IRD-')+4,99)))
                        WHEN {field} in ('NOT PROVIDED' , 'N/A') THEN cast({field} as nvarchar2(100))
                        ELSE fn_dc_method10_passport_hash({field})
                        END"""
                elif row['sub_method'] == 'method3':
                    sql = f"""{field} = CASE
		                WHEN {field} in ('NOT PROVIDED' , 'N/A') THEN {field}
		                WHEN dst.av_id_type_code = 'PASSPORT' THEN to_char(fn_dc_method10_passport_hash({field})    )
		                ELSE CONCAT(SUBSTR({field}, 1, LENGTH({field}) - LENGTH(SUBSTR({field}, -6))), COALESCE(src.id_mapping, SUBSTR({field}, -6)))
		                END"""

            elif row['hash_method'] == 'fn_dc_method14_hash':
                if pd.isna(row['sub_method']):
                    sql = f"{field} = CASE WHEN UPPER({field}) not in ('NOT PROVIDED' , 'N/A') THEN fn_dc_method14_hash({field}, m14s , m14t) ELSE cast({field} as nvarchar2(100)) END"
                elif row['sub_method'] == 'method1':
                    sql = f"{field} = CASE WHEN UPPER({field}) not in ('NOT PROVIDED' , 'N/A') THEN REGEXP_REPLACE(fn_dc_method14_hash({field}, m14s , m14t), '[^0-9a-zA-Z]','') ELSE cast({field}  as nvarchar2(100)) END"

            elif row['hash_method'] == 'fn_dc_method12_hash':
                if pd.isna(row['sub_method']):
                    sql = f"{field} = fn_dc_method12_hash({field}, m12)"

            elif row['hash_method'] == 'fn_dc_method16_hash':
                if pd.isna(row['sub_method']):
                    sql = f"{field} = fn_dc_method16_hash()"

            elif row['hash_method'] == 'fn_dc_method3_hash':
                if pd.isna(row['sub_method']):
                    sql = f"{field} = fn_dc_method3_hash({field}, m03)"

            elif row['hash_method'] == 'genCheckDigit_hash':
                if pd.isna(row['sub_method']):
                    sql = f"{field} = CASE WHEN {arg} IS NOT NULL THEN genCheckDigit_hash({arg}) END"
                elif row['sub_method'] == 'HKID':
                    sql = f"{field} = CASE WHEN dst.av_id_type_code='HKID' and {arg} IS NOT NULL  THEN genCheckDigit_hash({arg}) END"


            elif row['hash_method'] == 'Method2':
                if pd.isna(row['sub_method']):
                    sql = f"{field} = CASE WHEN {arg} IS NOT NULL THEN hashNumeric_hash({arg}, 0, m02) END"
                elif row['sub_method'] == 'method1':
                    sql = f"{field} = TO_NUMBER(hashNumeric_hash(TO_CHAR({arg}), 0, m02))"

            elif row['hash_method'] == 'Method17':
                sql = f"{field} = '{arg[4:]}'"
            elif row['hash_method'] == 'fn_dc_method13_hash':
                sql = f"{field} = fn_dc_method13_hash()"

            elif row['hash_method'] == 'fn_dc_method15_hash':
                if pd.isna(row['sub_method']):
                    sql = f"{field} = CASE WHEN {field} is null then null else fn_dc_method15_hash() END"
            sql_texts4.append(sql)

        if sql_texts4:
            list4.append((table, sql_texts4))

        # List 5: Updated to use translate_after_merge
        table_df2 = df2[df2['table'] == table]
        sql_texts5 = []

        for _, row in table_df2.iterrows():
            field = f"{row['field']}"

            if row['translate_after_merge'] == 'Y':
                sql = f"dst.{field} = translate(dst.{field}, eng_translate_src, eng_translate_tgt)"
            elif row['translate_after_merge'] == 'Y1':
                sql = f"dst.{field} = translate(substr(dst.{field},1,1), zhhk_translate_src, zhhk_translate_tgt)"
            elif row['translate_after_merge'] == 'Y2':
                sql = f"dst.{field} = translate(substr(dst.{field},2,2), zhhk_translate_src, zhhk_translate_tgt)"
            elif row['translate_after_merge'] == 'Y3':
                sql = f"dst.{field} = translate(substr(dst.{field},1,10), zhhk_translate_src, zhhk_translate_tgt)"
            if row['translate_after_merge'] in ['Y', 'Y1', 'Y2', 'Y3']:
                sql_texts5.append(sql)

        if sql_texts5:
            list5.append((table, sql_texts5))

        # List 6: New list for Update_after_Merge = Y
        table_df6 = df4[(df4['table'] == table) & (df4['update_after_merge'] == 'Y')]
        sql_texts6 = []
        sql_texts11 = []

        for _, row in table_df6.iterrows():
            field = f"dst.{row['field']}"
            arg = f"dst.{row['arg']}" if pd.notna(row['arg']) else None
            arg1 = f"src.{row['arg']}" if pd.notna(row['arg']) else None

            if row['hash_method'] == 'genCheckDigit_hash':
                if pd.isna(row['sub_method']):
                    sql = f"{field} = CASE WHEN {arg} IS NOT NULL THEN genCheckDigit_hash({arg}) END"
                elif row['sub_method'] == 'HKID':
                    sql = f"{field} = CASE WHEN av_id_type_code='HKID' AND  (LENGTH({arg})  =7 or LENGTH({arg})  =8) THEN genCheckDigit_hash({arg}) END"
                sql_texts6.append(sql)
            elif row['hash_method'] == 'fn_dc_method10_passport_hash':
                if pd.isna(row['sub_method']):
                    sql = f"{field} = CASE WHEN {field} IS NOT NULL AND {field} <> 'N/A' THEN fn_dc_method10_passport_hash({arg}) END"
                elif row['sub_method'] == 'method1':
                    sql = f"""{field} = CASE
                        WHEN {field} like '%-PASSPORT-%' THEN CONCAT(substr({field},1, instr({field},'-PASSPORT-')+9) , fn_dc_method10_passport_hash(substr({field},instr({field},'-PASSPORT-')+10,99)))
                        WHEN SUBSTR({field}, 1, 9) = 'PASSPORT-' THEN CONCAT(SUBSTR({field}, 1, 9), fn_dc_method10_passport_hash(SUBSTR({field}, 10, LENGTH({field})-9)))
                        WHEN SUBSTR({field}, 1, 5) = 'HKID-' THEN CONCAT(SUBSTR({field}, 1, 5), fn_dc_method10_passport_hash(SUBSTR({field}, 6, LENGTH({field})-5)))
                        WHEN {field} like '%HKID-%' THEN substr({field},0, instr({field},'HKID-')+4) || rtrim(fn_dc_method10_passport_hash(substr({field}, instr({field},'HKID-')+5,6)))||
                            substr(use_customer_no, instr({field},'HKID-')+11,99)
                        WHEN {field} in ('NOT PROVIDED' , 'N/A') THEN cast({field} as nvarchar2(100))
                        ELSE fn_dc_method10_passport_hash({field})
                        END"""
                elif row['sub_method'] == 'method2':
                    sql = f"""{field} = CASE
				    WHEN {field} in ('NOT PROVIDED' , 'N/A') THEN cast({field} as nvarchar2(100)) 										WHEN av_id_type_code='PASSPORT' THEN fn_dc_method10_passport_hash({field})
				    WHEN av_id_type_code <> 'HKID' AND 	av_id_type_code <> 'PASSPORT' THEN fn_dc_method14_hash({field}, m14s , m14t)
				    WHEN av_id_type_code = 'HKID' THEN TO_NCHAR({field})
				    ELSE fn_dc_method10_passport_hash({field})
			        END"""
                sql_texts6.append(sql)
            elif row['hash_method'] == 'fn_dc_method16_hash':
                if pd.isna(row['sub_method']):
                    sql = f"{field} = fn_dc_method16_hash()"
                sql_texts6.append(sql)
            elif row['hash_method'] == 'Method17':
                sql = f"{field} = '{arg[4:]}'"
                sql_texts6.append(sql)
            elif row['hash_method'] == 'fn_dc_method3_hash':
                if pd.isna(row['sub_method']):
                    sql = f"{field} = fn_dc_method3_hash({arg}, m03)"
                sql_texts6.append(sql)
            elif row['hash_method'] == 'fn_dc_method14_hash':
                if pd.isna(row['sub_method']):
                    sql = f"{field} = CASE WHEN UPPER({field}) not in ('NOT PROVIDED' , 'N/A') THEN fn_dc_method14_hash({arg}, m14s , m14t) ELSE cast({arg} as nvarchar2(100)) END"
                sql_texts6.append(sql)
            elif row['hash_method'] == 'fn_dc_method12_hash':
                if pd.isna(row['sub_method']):
                    sql = f"{field} = fn_dc_method12_hash({field}, m12)"
                sql_texts6.append(sql)
            elif row['hash_method'] == 'Special':
                if pd.isna(row['sub_method']):
                    sql = f"{field} = substr({arg1},-3,3)"
                sql_texts11.append(sql)
            else:
                sql_texts6.append(f"""{field} = {row['hash_method']}({arg})""")


        if sql_texts6:
            list6.append((table, sql_texts6))
        if sql_texts11:
            list11.append((table, sql_texts11))
        # List 7 & 8: Get update_where and merge_where conditions
        table_df5 = df5[df5['table'] == table]

        # List 7: Update where conditions
        update_where_conditions = table_df5['update_where'].dropna().tolist()
        if update_where_conditions:
            list7.append((table, update_where_conditions))

        # List 8: Merge where conditions
        merge_where_conditions = table_df5['merge_where'].dropna().tolist()
        if merge_where_conditions:
            list8.append((table, merge_where_conditions))

        # List 9: Update after where conditions
        table_df6 = df6[df6['table'] == table]
        update_after_where_conditions = table_df6['update_after_where'].dropna().tolist()
        if update_after_where_conditions:
            list9.append((table, update_after_where_conditions))

    # Print the resulting lists
    print("\nList 1 (Table with Fields):")
    for item in list1:
        print(f"\nTable: {item[0]}")
        print("Fields:", item[1])

    print("\nList 2 (Table with SQL):")
    for item in list2:
        print(f"\nTable: {item[0]}")
        print("SQL statements:")
        for sql in item[1]:
            print(sql)
            print()

    print("\nList 3 (Table with SQL):")
    for item in list3:
        print(f"\nTable: {item[0]}")
        print("SQL statements:")
        for sql in item[1]:
            print(sql)
            print()

    print("\nList 4 (Table with SQL):")
    for item in list4:
        print(f"\nTable: {item[0]}")
        print("SQL statements:")
        # Create a dictionary to count occurrences
        for sql in item[1]:
            print(sql)
            print()

    print("\nList 5 (Table with Translation SQL):")
    for item in list5:
        print(f"\nTable: {item[0]}")
        print("SQL statements:")
        for sql in item[1]:
            print(sql)
            print()

    print("\nList 6 (Table with Translation SQL):")
    for item in list6:
        print(f"\nTable: {item[0]}")
        print("SQL statements:")
        for sql in item[1]:
            print(sql)
            print()

    print("\nList 7 (Table with SQL):")
    for item in list7:
        print(f"\nTable: {item[0]}")
        print("Update where conditions:")
        for sql in item[1]:
            print(sql)


    print("\nList 8 (Table with Where Conditions):")
    for item in list8:
        print(f"\nTable: {item[0]}")
        print("Merge where conditions:")
        for condition in item[1]:
            print(condition)

    print("\nList 9 (Table with Where Conditions):")
    for item in list9:
        print(f"\nTable: {item[0]}")
        print("Update after where conditions:")
        for condition in item[1]:
            print(condition)

    print("\nList 11 (Table with SQL):")
    for item in list11:
        print(f"\nTable: {item[0]}")
        print("SQL statements:")
        for sql in item[1]:
            print(sql)

    def generate_final_sql(list1, list2, list3, list4, list5, list6, list7, list8, list9):
        final_list = []



        # Process each table
        for table in df1['table'].unique():
            sql_parts = []
            cross_apply_list = next((item[1] for item in list2 if item[0] == table), [])
            left_join_list = next((item[1] for item in list3 if item[0] == table), [])
            if cross_apply_list or left_join_list:
            # Section 1
                fields_list = next((item[1] for item in list1 if item[0] == table), [])
                section1 = f"""
                ---------------
                -- {table}
                ---------------
                dbms_output.put_line('-- hashing {table} --');
                MERGE INTO {table} dst
                USING (
                    SELECT
                        {',\n                '.join(fields_list)}
                    FROM {table} org"""
                sql_parts.append(section1)
                if cross_apply_list:
                    section2 = '\n'.join(f"""
                    CROSS APPLY (
                        {ca}
                    )""" for ca in cross_apply_list)
                    sql_parts.append(section2)
                if left_join_list:
                    section2_1 = '\n'.join(f"""
                    OUTER APPLY (
                        {lf}
                    )""" for lf in left_join_list)
                    sql_parts.append(section2_1)

            # Section 3
          #  where_conditions = next((item[1] for item in list3 if item[0] == table), [])
                section3 = """
                    WHERE org.SCHEME_CODE = schemeAbbreviation
                    """ + '\n                    '.join([f"AND {condition}" for condition in next((item[1] for item in list8 if item[0] == table), [])]) + f"""
                ) src
                ON (dst.ID = src.ID)
                WHEN MATCHED THEN UPDATE SET"""
                sql_parts.append(section3)
                update_statements = next((item[1] for item in list4 if item[0] == table), [])
                section4 = f"""            {',\n            '.join(update_statements)};
                recCount := SQL%rowcount;
                dbms_output.put_line('.  {table}(id, name)=' || recCount);"""
                sql_parts.append(section4)
            else:
                section0 = f"""
                ---------------
                -- {table}
                ---------------
                dbms_output.put_line('-- hashing {table} --');
                UPDATE {table} dst
                SET
                """
                sql_parts.append(section0)
                update_statements = next((item[1] for item in list4 if item[0] == table), [])
                section4 = f"""            {',\n            '.join(update_statements)}
                WHERE SCHEME_CODE = schemeAbbreviation """ + '\n            '.join([f"AND {condition}" for condition in next((item[1] for item in list7 if item[0] == table), [])]) + f""";
                recCount := SQL%rowcount;
                dbms_output.put_line('.  {table}(id, name)=' || recCount);"""
                sql_parts.append(section4)

            # Section 5 (optional)
            translate_statements = next((item[1] for item in list5 if item[0] == table), [])
            if translate_statements:
                section5 = f"""
        UPDATE {table} dst
        SET
            {',\n            '.join(translate_statements)}
        WHERE SCHEME_CODE = schemeAbbreviation;

        recCount := SQL%rowcount;
        dbms_output.put_line('.  {table}(translate)=' || recCount);"""
                sql_parts.append(section5)

            update_statements = next((item[1] for item in list6 if item[0] == table), [])
            if update_statements:
                section6 = f"""
        UPDATE {table} dst
        SET
            {',\n        '.join(update_statements)}
        WHERE SCHEME_CODE = schemeAbbreviation """ + '\n            '.join([f"AND {condition}" for condition in next((item[1] for item in list9 if item[0] == table), [])]) + f""";
        recCount:=SQL%rowcount;
        dbms_output.put_line('.  {table}(update)=' || recCount);"""
                sql_parts.append(section6)

            special_update_statements = next((item[1] for item in list11 if item[0] == table), [])
            if special_update_statements:
                section11 = f"""
        MERGE INTO {table} dst
        USING
			(
				SELECT
				doc.client_uuid,
				doc.ID_no
				FROM
				CMN_CLIENT_ID_DOC doc
				LEFT JOIN
				REG_EMPLOYER er on (doc.client_uuid = er.id)
				WHERE
					doc.AV_ID_TYPE_CODE ='BR' and doc.ID_NO IS NOT NULL
				) src
			ON
			(
				src.client_uuid = dst.ID
			)
			WHEN MATCHED THEN UPDATE
			SET
            {',\n        '.join(special_update_statements)};
        recCount:=SQL%rowcount;
        dbms_output.put_line('.  {table}(update)=' || recCount);"""
                sql_parts.append(section11)





            # Combine all sections
            final_sql = '\n'.join(sql_parts)
            final_list.append((table, final_sql))



        # Print the final generated SQL for each table
        print("\nFinal Generated SQL:")

        # Export to file
        with open('hashing.sql', 'w') as f:
            # Write header with consistent indentation
            f.write("""DECLARE
    recCount            NUMBER;
    schemeAbbreviation VARCHAR2(10)  := 'XXXX';
    m02                VARCHAR2(100) := 'XXXX';
    m03                VARCHAR2(100) := 'XXXX';
    m11                VARCHAR2(100) := 'XXXX';
    m12                VARCHAR2(100) := 'XXXX';
    m14s               VARCHAR2(100) := 'XXXX';
    m14t               VARCHAR2(100) := 'XXXX';
    eng_translate_src  VARCHAR2(100) := 'XXXX';
    eng_translate_tgt  VARCHAR2(100) := 'XXXX';
    zhhk_translate_src VARCHAR2(100) := 'XXXX';
    zhhk_translate_tgt VARCHAR2(100) := 'XXXX';
BEGIN""")

            for table, sql in final_list:
                # Add newlines between each table's SQL block
                f.write(f"\n\n{sql}")

            f.write("""

END;
/""")

        return final_list

    # Add this call after all lists are created
    final_sql_list = generate_final_sql(list1, list2, list3, list4, list5, list6, list7,list8,list9)

# Usage
if __name__ == "__main__":
    file_path = "Sample1.csv"
    process_hash_config(file_path)



Dataframe 1 (Table, Field):
                    table            field include_in_select
0          cmn_aml_entity       biz_reg_no                 Y
1          cmn_aml_entity  cert_of_corp_no                 Y
2          cmn_aml_entity          co_name               NaN
3          cmn_aml_entity     co_name_zhhk               NaN
4          cmn_aml_entity  use_customer_no                 Y
..                    ...              ...               ...
213             DM_MEMBER       OCCUPATION               NaN
214             DM_MEMBER       STAFF_NMBR               NaN
215             DM_MEMBER  FORMER_EMPLR_NM               NaN
216  DM_MEMBER_SUPPLEMENT       STAFF_NMBR               NaN
217                   NaN              NaN               NaN

[217 rows x 3 columns]

Dataframe 2 (Table, Field, Translate_after_merge):
                    table            field translate_after_merge
0          cmn_aml_entity       biz_reg_no                   NaN
1          cmn_aml_entity  cert_o